In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re

In [2]:
#data loader class
class DataLoader:
    def __init__(self,dir_name,file_name):
        self.dir_name=dir_name
        self.file_name = file_name
    
 
    def read_csv(self):
        os.chdir(self.dir_name)
        tweets_df=pd.read_csv(self.file_name)
        return tweets_df
  

In [13]:
#object creation
DataLoader_obj= DataLoader('../data/','cleaned_tweet_data.csv')

In [14]:
tweets_df=DataLoader_obj.read_csv()
tweets_df.dropna()

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,clean_text


In [15]:
len(tweets_df)

0

In [16]:
tweets_df.head()

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place,clean_text


In [17]:
class PrepareData:
    def __init__(self,df):
        self.df=df
    
    def preprocess_data(self):
        tweets_df = self.df.loc[self.df['lang'] =="en"]


        #text Preprocessing
        tweets_df['clean_text']=tweets_df['clean_text'].astype(str)
        tweets_df['clean_text'] = tweets_df['clean_text'].apply(lambda x: x.lower())
        tweets_df['clean_text']= tweets_df['clean_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))

        #Converting tweets to list of words For feature engineering
        sentence_list = [tweet for tweet in tweets_df['clean_text']]
        word_list = [sent.split() for sent in sentence_list]
        # print(word_list)

        #Create dictionary which contains Id and word 
        word_to_id = corpora.Dictionary(word_list) #generate unique tokens
        #  we can see the word to unique integer mapping
        # print(word_to_id.token2id)
        # using bag of words(bow), we create a corpus that contains the word id and its frequency in each document.
        corpus_1= [word_to_id.doc2bow(tweet) for tweet in word_list]


        return word_list, word_to_id, corpus_1


In [18]:
PrepareData_obj=PrepareData(tweets_df)
word_list ,id2word,corpus=PrepareData_obj.preprocess_data()